# CARTOON MODEL
 
Utilizamos el modelo CartoonGAN, creado por un equipo de investigadores incluyendo la Universidad Tsinghua en China y la Universidad de Cardiff en el Reino Unido.

Nuestro código, basado en el realizado por YANG ZHOU, toma una foto en formato digital y la preprocesa adaptandola al tamaño adecuado y realiza los ajustes necesarios para que el modelo pueda procesarla. En segundo lugar, aplica CartoonGAN, para ello utiliza el archivo preentrenado que contiene el modelo y la foto se convierte en una imagen con el estilo de caricatura. Finalmente, la imagen procesada se guarda en un directorio designado, lista para ser usada o compartida.

In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Función para cargar una imagen
def load_image(img_path):
    img = cv2.imread(img_path)
    img = img.astype(np.float32) / 127.5 - 1
    img = np.expand_dims(img, 0)
    img = tf.convert_to_tensor(img)
    return img

# Función para preprocesar una imagen
def preprocess_image(img, target_dim=224):
    shape = tf.cast(tf.shape(img)[1:-1], tf.float32)
    smallest_dim = min(shape)
    scale = target_dim / smallest_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = tf.image.resize_with_crop_or_pad(img, target_dim, target_dim)
    return img

# Función para procesar todas las imágenes en un directorio
def process_images(input_dir, output_dir):
    # Cargando el modelo
    base_path = os.getcwd()
    model_filename = '1.tflite'
    model_path = os.path.join(base_path, 'cartoongan', 'tfLite', 'dr', '1', model_filename)
    
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    # Procesando cada imagen en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}-out.jpg")  # Cambiar a .png si es necesario

            input_image = load_image(input_path)
            preprocessed_image = preprocess_image(input_image, target_dim=512)

            interpreter.set_tensor(interpreter.get_input_details()[0]['index'], preprocessed_image)
            interpreter.invoke()

            output_tensor = interpreter.tensor(interpreter.get_output_details()[0]['index'])()
            output_image = (np.squeeze(output_tensor) + 1.0) * 127.5
            output_image = np.clip(output_image, 0, 255).astype(np.uint8)
            output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB)

            # Guardar la imagen y liberar referencias
            cv2.imwrite(output_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))
            output_image = None
            output_tensor = None
            preprocessed_image = None
            input_image = None
            
            print(f'Imagen procesada y guardada en: {output_path}')

# Directorios de entrada y salida
input_directory = "E://NEURONAL//cartoongan//I"
output_directory = "E://NEURONAL//cartoongan//O"

# Crear el directorio de salida si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Procesar las imágenes en el directorio de entrada y guardarlas en el directorio de salida
process_images(input_directory, output_directory)



Imagen procesada y guardada en: E://NEURONAL//cartoongan//O\25-out.jpg
Imagen procesada y guardada en: E://NEURONAL//cartoongan//O\HITA-out.jpg
Imagen procesada y guardada en: E://NEURONAL//cartoongan//O\RT3-out.jpg
Imagen procesada y guardada en: E://NEURONAL//cartoongan//O\SG-out.jpg
